Question 7:  Using the likelihood ratios provided, calculate the probability of mortality for the cases described in the case file. In the attached likelihood ratio file, ICD9 variable refers to the diagnosis code, RDX refers to the first, second, third, fourth or fifth time the diagnosis code occurs for the same person. LR is the calculated likelihood ratio for a specific diagnosis and repeat code.  A likelihood ratio above 1 indicates the person is more likely to die. The variable nDead refers to number of patients with specific diagnosis and repeat code dying in 6 months, nAlive refers to number of patients with specific diagnosis and repeat code being alive in 6 months. Using the case file, predict the probability of death in 6 months for cases with different medical history. List the 4 patients with the highest probability of mortality. 

In [1]:
import pandas as pd

In [2]:
import numpy as np 

In [8]:
#Importing the Likelihood Ratio file 

lrfile = pd.read_excel(r'C:\Users\seble\OneDrive\Desktop\GMU Fall 2022\HAP 725\4. Risk Assessment\Likelihood Ratios for Predicting Prognosis.xlsx')

In [7]:
lrfile.head()

,icd9,RDX,LR,nDead,nAlive,TotalDead,TotalNotDead
0,I001.0,1,0.333333,0,2,121567,611807
1,I001.9,1,0.333333,0,2,121567,611807
2,I002.0,1,0.500000,0,1,121567,611807
3,I003.0,1,0.141765,4,142,121567,611807
4,I003.0,2,0.500000,0,1,121567,611807


In [9]:
#Indicate the number of rows and columns in the lrfile dataframe 
print(lrfile.shape)

(24598, 7)


In [12]:
#Import the case file 
casefile = pd.read_csv(r'C:\Users\seble\OneDrive\Desktop\GMU Fall 2022\HAP 725\4. Risk Assessment\CaseData.csv')

In [13]:
#Indicate the number of rows and columns in the casefile dataframe
print(casefile.shape)

(378, 4)


In [14]:
#Limit the number of columns to use from the casefile 
caselim = casefile[['id','icd9','AgeAtDx']]

In [16]:
#Indicate the number of rows and columns in caseslim 
print(caselim.shape)

(378, 3)


In [21]:
#Group by the caselim dataframe by using the id and icd9 columns 
grouped = caselim.groupby(['id','icd9'], as_index=False)['AgeAtDx'].count()
grouped2 = pd.DataFrame(grouped)

#Rename the column names 
grprename = grouped2.rename(columns={"id":"id","icd9":"icd9","AgeAtDx":"RDX"})
grouped3 = pd.DataFrame(grprename)

#print dataframe 
grouped3

,id,icd9,RDX
0,19980,I276.51,1
1,19980,I300.00,1
2,19980,I458.0,1
3,19980,I558.9,1
4,19980,I780.52,1
...,...,...,...
256,20000,I303.91,1
257,20000,I304.31,1
258,20000,I309.81,2
259,20000,I719.49,1


In [22]:
#show the count of the various attriutes in the grouped3 df 
grouped.count()

id         261
icd9       261
AgeAtDx    261
dtype: int64

In [23]:
#shape, this indicates the number of rows and columns in the grouped3 dataframe 
print(grouped3.shape)

(261, 3)


In [24]:
#shows unique values in each of the columns of the grouped3 df 
grouped3.nunique()

id       21
icd9    184
RDX       6
dtype: int64

In [25]:
#Display column names from the lrfile df 
lrfile.columns 

Index(['icd9', 'RDX', 'LR', 'nDead', 'nAlive', 'TotalDead', 'TotalNotDead'], dtype='object')

In [26]:
#Display column names from the grouped3 df
grouped3.columns

Index(['id', 'icd9', 'RDX'], dtype='object')

In [27]:
#Join the grouped3 and lrfile dataframes
merged = pd.merge(grouped,lrfile)

#Display column names from the merged df 
merged.columns 

Index(['id', 'icd9', 'AgeAtDx', 'RDX', 'LR', 'nDead', 'nAlive', 'TotalDead',
       'TotalNotDead'],
      dtype='object')

In [28]:
#Finetune by using left join on icd9 and RDX columns 
merged = pd.merge(grouped3, lrfile, on = ('icd9','RDX'), how='left')

#Show top 5 rows of the merged data frame
merged.head(5)

,id,icd9,RDX,LR,nDead,nAlive,TotalDead,TotalNotDead
0,19980,I276.51,1,0.680183,3975.0,29411.0,121567.0,611807.0
1,19980,I300.00,1,0.304057,2619.0,43349.0,121567.0,611807.0
2,19980,I458.0,1,0.437280,1216.0,13995.0,121567.0,611807.0
3,19980,I558.9,1,0.273063,539.0,9934.0,121567.0,611807.0
4,19980,I780.52,1,0.240356,1381.0,28916.0,121567.0,611807.0


In [31]:
#Find the LOG of LR for all Patient instances 

merged['LogPTS']=np.log(merged['LR'])

#Show top 5 rows of a the Merged data frame
merged.head()

,id,icd9,RDX,LR,nDead,nAlive,TotalDead,TotalNotDead,LogPTS
0,19980,I276.51,1,0.680183,3975.0,29411.0,121567.0,611807.0,-0.385393
1,19980,I300.00,1,0.304057,2619.0,43349.0,121567.0,611807.0,-1.190540
2,19980,I458.0,1,0.437280,1216.0,13995.0,121567.0,611807.0,-0.827182
3,19980,I558.9,1,0.273063,539.0,9934.0,121567.0,611807.0,-1.298052
4,19980,I780.52,1,0.240356,1381.0,28916.0,121567.0,611807.0,-1.425636


In [32]:
#Find the sum of the Log for each Patient grouped by patient ID and log of likelihood ratio 
merged2 = merged.groupby('id')['LogPTS'].sum().reset_index()

#Put merged2 output data into dataframe
merged2_df = pd.DataFrame(merged2)

#print dataframe 
merged2_df

,id,LogPTS
0,19980,-5.126802
1,19981,-44.024474
2,19982,-8.485988
3,19983,-8.540649
4,19984,-11.263853
5,19985,-16.187559
6,19986,-6.374092
7,19987,-11.119499
8,19988,-25.326099
9,19989,-8.027352


In [33]:
#Find the Odds for each patient 
merged2_df['OddsPTS']=np.exp(merged2_df['LogPTS'])
merged2_df2 = pd.DataFrame(merged2_df)

#print dataframe
merged2_df2

,id,LogPTS,OddsPTS
0,19980,-5.126802,5.935512e-03
1,19981,-44.024474,7.593011e-20
2,19982,-8.485988,2.063394e-04
3,19983,-8.540649,1.953634e-04
4,19984,-11.263853,1.282836e-05
5,19985,-16.187559,9.328942e-08
6,19986,-6.374092,1.705168e-03
7,19987,-11.119499,1.482050e-05
8,19988,-25.326099,1.002340e-11
9,19989,-8.027352,3.264114e-04


In [35]:
#Determine the probability for each patient rounded to 2 decimal places 

merged2_df2['Probability']=((merged2_df2['OddsPTS'])/(1+(merged2_df2['OddsPTS']))).round(decimals=2)
merged2_prob = pd.DataFrame(merged2_df2)

#print dataframe 
merged2_prob 

#sort values by probability descending 
merged2_prob.sort_values(by='Probability',ascending=False)

,id,LogPTS,OddsPTS,Probability
13,19993,-0.726423,4.836359e-01,0.33
15,19995,-4.029526,1.778276e-02,0.02
0,19980,-5.126802,5.935512e-03,0.01
12,19992,-5.145156,5.827567e-03,0.01
11,19991,-31.547590,1.990929e-14,0.00
19,19999,-25.286040,1.043308e-11,0.00
18,19998,-14.968653,3.156435e-07,0.00
17,19997,-22.604919,1.523381e-10,0.00
16,19996,-49.487960,3.218491e-22,0.00
14,19994,-8.969134,1.272784e-04,0.00
